# Importações

In [1]:
from faker import Faker
import random
from datetime import datetime
import sys
import os

In [2]:
# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [3]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

## Funções Gerais

In [4]:
# Dicionário para armazenar os DREs gerados por ano e semestre sem repetição
DRE_REGISTRO = {}

def gerar_dre(periodo_ingresso_ufrj):
    periodo, semestre = periodo_ingresso_ufrj.split('/')
    periodo = periodo[-2:]
    semestre = str(int(semestre)-1)
    
    chave = f'{periodo}{semestre}'
    if chave not in DRE_REGISTRO:
        DRE_REGISTRO[chave] = set()
    
    while True:
        ultimos_digitos = f'{random.randint(0, 99999):05d}'
        dre = f'1{periodo}{semestre}{ultimos_digitos}'
        if ultimos_digitos not in DRE_REGISTRO[chave]:
            DRE_REGISTRO[chave].add(ultimos_digitos)
            return dre

In [5]:
def calcular_idade_ingresso_curso_atual(row):
    """
    Calcula a idade atual com base na data de nascimento fornecida.
    
    Args:
    data_nascimento (str): Data de nascimento no formato 'dd/mm/yyyy'.
    
    Returns:
    int: Idade atual em anos, ou None se a data estiver em um formato inválido.
    """
    data_nascimento = row['dataNascimento']
    periodo_ingresso = row['periodoIngressoCursoAtual']
    try:
        # Converter a string da data de nascimento em um objeto datetime
        ano_nascimento = datetime.strptime(data_nascimento, '%d/%m/%Y').year

        # Obter a data atual
        ano_ingresso = int(periodo_ingresso.split('/')[0])

        # Calcular a idade
        idade = ano_ingresso - ano_nascimento
        return idade
    except ValueError:
        # Retornar None se a data de nascimento estiver em um formato inválido
        return None

# Processamento


Algumas colunas não trazem ganho de informação, como por exemplo reprovações já estão na coluna disciplinas cursadas, e periodo com cr menor que 3, pode ser analisar na coluna CR por periodo


In [6]:
df_bronze = extrair_dados_database("alunos_ufrj.parquet", "bronze")

In [8]:
# Inicializa o Faker para gerar nomes em português do Brasil
faker = Faker("pt_BR")

# Gerar um nome e DRE ficticios para cada aluno
df_bronze["nomeCompleto"] = [faker.name() for _ in range(df_bronze.shape[0])]
df_bronze["matriculaDre"] = df_bronze['periodoIngressoUFRJ'].apply(gerar_dre)

# Calcula a idade de quando o aluno ingressou no curso Atual
df_bronze['idadeIngressoCursoAtual'] = df_bronze.apply(calcular_idade_ingresso_curso_atual, axis=1)


In [9]:
# Filtra somente por alunos que estao no curso da CC
lista_cod_curso_computacao = [3101020000, 3101070000, 3109000100]
df_bronze = df_bronze[df_bronze["codCursoAtual"].isin(lista_cod_curso_computacao)]

In [10]:
# Normalizando a coluna sexo
mapeamento_genero = {
    'M': 'Masculino',
    'F': 'Feminino'
}
df_bronze['sexo'] = df_bronze['sexo'].map(mapeamento_genero)

In [11]:
# Salvar dados na camada Silver
carregar_dados_database(df_bronze, "alunos_ufrj.parquet", "silver")